# 如何转换子图的输入和输出

您的子图状态可能完全独立于父图状态，即两者之间没有重叠的通道（键）。例如，您可能有一个主管代理，需要在多个 ReAct 代理的帮助下生成报告。ReAct 代理子图可能会跟踪消息列表，而主管只需要用户输入和其状态的最终报告，并且不需要跟踪消息。

在这种情况下，您需要在调用子图之前将输入转换为子图，然后在返回之前转换其输出。本指南展示了如何做到这一点。

## 设置

首先，我们安装所需的软件包
```bash
npm install @langchain/langgraph @langchain/core
```
<div class="警告提示">
<p class="admonition-title">设置 <a href="https://smith.langchain.com">LangSmith</a> 进行 LangGraph 开发</p>
<p style="padding-top: 5px;">
注册 LangSmith 以快速发现问题并提高 LangGraph 项目的性能。LangSmith 允许您使用跟踪数据来调试、测试和监控使用 LangGraph 构建的 LLM 应用程序 - 在<a href="https://docs.smith.langchain.com">此处</a>了解有关如何开始的更多信息。
</p>
</div>

## 定义图和子图

让我们定义 3 个图：
- 父图
- 将由父图调用的子子图
- 将由子图调用的孙子子图

### 定义孙子

In [1]:
import { StateGraph, START, Annotation } from "@langchain/langgraph";

const GrandChildAnnotation = Annotation.Root({
    myGrandchildKey: Annotation<string>,
})

const grandchild1 = (state: typeof GrandChildAnnotation.State) => {
    // 注意：此处无法访问子键或父键
    return {
        myGrandchildKey: state.myGrandchildKey + ", how are you"
    }
}

const grandchild = new StateGraph(GrandChildAnnotation)
    .addNode("grandchild1", grandchild1)
    .addEdge(START, "grandchild1")

const grandchildGraph = grandchild.compile();

In [2]:
await grandchildGraph.invoke({ myGrandchildKey: "hi Bob" })

{ myGrandchildKey: 'hi Bob, how are you' }


### 定义孩子

In [5]:
import { StateGraph, START, Annotation } from "@langchain/langgraph";

const ChildAnnotation = Annotation.Root({
    myChildKey: Annotation<string>,
});

const callGrandchildGraph = async (state: typeof ChildAnnotation.State) => {
    // 注意：此处无法访问父级或孙级密钥
    // 我们正在从子状态通道（`myChildKey`）转换状态
    // 到孙子状态通道（`myGrandchildKey`）
    const grandchildGraphInput = { myGrandchildKey: state.myChildKey };
    // 我们正在从孙子状态通道（`myGrandchildKey`）转换状态
    // 返回子状态通道（`myChildKey`）
    const grandchildGraphOutput = await grandchildGraph.invoke(grandchildGraphInput);
    return {
        myChildKey: grandchildGraphOutput.myGrandchildKey + " today?"
    };
};

const child = new StateGraph(ChildAnnotation)
    // 注意：我们在这里传递一个函数，而不仅仅是编译后的图（`childGraph`）
    .addNode("child1", callGrandchildGraph)
    .addEdge(START, "child1");

const childGraph = child.compile();

In [6]:
await childGraph.invoke({ myChildKey: "hi Bob" })

{ myChildKey: 'hi Bob, how are you today?' }


<div class="警告信息">
<p class="admonition-title">注意</p>
<p>
我们将 <code>grandchildGraph</code> 调用包装在一个单独的函数 (<code>callGrandchildGraph</code>) 中，该函数在调用孙子图之前转换输入状态，然后将孙子图的输出转换回子图状态。如果您只是将 <code>grandchildGraph</code> 直接传递给 <code>.addNode</code> 而不进行转换，LangGraph 将引发错误，因为子状态和孙状态之间没有共享状态通道（键）。
</p>
</div>

请注意，子子图和孙子子图有自己的**独立**状态，不与父图共享。

### 定义父级

In [7]:
import { StateGraph, START, END, Annotation } from "@langchain/langgraph";

const ParentAnnotation = Annotation.Root({
    myKey: Annotation<string>,
});

const parent1 = (state: typeof ParentAnnotation.State) => {
    // 注意：此处无法访问子键或孙键
    return { myKey: "hi " + state.myKey };
};

const parent2 = (state: typeof ParentAnnotation.State) => {
    return { myKey: state.myKey + " bye!" };
};

const callChildGraph = async (state: typeof ParentAnnotation.State) => {
    // 我们正在从父状态通道（`myKey`）转换状态
    // 到子状态通道（`myChildKey`）
    const childGraphInput = { myChildKey: state.myKey };
    // 我们正在从子状态通道（`myChildKey`）转换状态
    // 返回父状态通道（`myKey`）
    const childGraphOutput = await childGraph.invoke(childGraphInput);
    return { myKey: childGraphOutput.myChildKey };
};

const parent = new StateGraph(ParentAnnotation)
    .addNode("parent1", parent1)
    // 注意：我们在这里传递一个函数，而不仅仅是一个编译图（`childGraph`）
    .addNode("child", callChildGraph)
    .addNode("parent2", parent2)
    .addEdge(START, "parent1")
    .addEdge("parent1", "child")
    .addEdge("child", "parent2")
    .addEdge("parent2", END);

const parentGraph = parent.compile();

<div class="警告信息">
<p class="admonition-title">注意</p>
<p>
我们将 <code>childGraph</code> 调用包装在一个单独的函数 (<code>callChildGraph</code>) 中，该函数在调用子图之前转换输入状态，然后将子图的输出转换回父图状态。如果您只是将 <code>childGraph</code> 直接传递给 <code>.addNode</code> 而不进行转换，LangGraph 将引发错误，因为父状态和子状态之间没有共享状态通道（键）。
</p>
</div>

让我们运行父图并确保它正确调用子子图和孙子子图：

In [8]:
await parentGraph.invoke({ myKey: "Bob" })

{ myKey: 'hi Bob, how are you today? bye!' }


完美的！父图正确调用子图和孙子图（我们知道，因为“你好吗”和“今天？”被添加到我们原始的“myKey”状态值中）。